In [1]:
from tensorflow.keras import utils
from tensorflow.keras.layers import InputLayer, Dense, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [4]:
# reshape c/ dimensão dos canais de cores para ficar de acordo com os moldes do tensorflow:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# casting de inteiro pra float para posterior normalização:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalização (p/ [0, 1]) para tornar os cálculos da rede neural menos custosos:
X_train /= 255
X_test /= 255

# transformação dos conjuntos de classe para codificação One Hot:
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

In [5]:
network = Sequential()
network.add(InputLayer(shape = (28, 28, 1)))
network.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size = (2, 2)))
network.add(Flatten())
network.add(Dense(units = 128, activation = 'relu'))
network.add(Dense(units = 10, activation = 'softmax'))

network.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [6]:
datagen_train = ImageDataGenerator(rotation_range = 7, horizontal_flip = True,
                            shear_range = 0.2, height_shift_range = 0.07,
                            zoom_range = 0.2)

Na célula acima, uma instância de ImageDataGenerator é gerada. Esse objeto será um transformador, que aplicará mudanças a conjuntos de imagens de acordo com o especificado em seus parâmetros.<br>
* rotation_range: faixa de valores, em graus, nos quais as imagens podem ser rotacionadas;
* horizontal_flip: permite a inversão horizontal de cada imagem;
* shear_range: proporção na qual a imagem pode sofrer o processo de "shearing", que é o deslocamento de linhas e colunas, mudando a forma da imagem;
* height_shift_range: proporção com a qual a imagem pode ser deslocada para cima ou para baixo;
* zoom_range: proporção com a qual podem ser feitos zooms nas imagens.

In [7]:
datagen_test = ImageDataGenerator()

O objeto acima servirá apenas para gerar a base de teste no mesmo formato (de gerador) que a base de treinamento. Mas nenhuma transformação será aplicada, já que isso não faz sentido para base de teste.

In [8]:
train_ds = datagen_train.flow(X_train, y_train, batch_size = 128) # retorna um gerador que gera batches de 128 pares de treino

In [10]:
test_ds = datagen_test.flow(X_test, y_test, batch_size = 128) # retorna um gerador que gera batches de 128 pares de teste

In [11]:
network.fit(train_ds, epochs = 5, validation_data = test_ds)

Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 80ms/step - accuracy: 0.8306 - loss: 0.5171 - val_accuracy: 0.9609 - val_loss: 0.2446
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 78ms/step - accuracy: 0.9545 - loss: 0.1464 - val_accuracy: 0.9686 - val_loss: 0.1005
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 78ms/step - accuracy: 0.9628 - loss: 0.1122 - val_accuracy: 0.9654 - val_loss: 0.1101
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 79ms/step - accuracy: 0.9701 - loss: 0.0954 - val_accuracy: 0.9779 - val_loss: 0.0725
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 78ms/step - accuracy: 0.9741 - loss: 0.0845 - val_accuracy: 0.9733 - val_loss: 0.0904
